In [1]:
import numpy as np
from copy import deepcopy
from math import sqrt

In [141]:
def random_matrix_size_n(n):
    return [[np.float32(np.random.uniform(0, 1)) for _ in range(n)] for _ in range(n)]

def random_vector_size_n(n):
    return [np.float32(np.random.uniform(0, 1))  for _ in range(n)]

In [3]:
def gauss_jordan(A, B):
    n = len(A)
    for i in range(n):
        for j in range(n):
            if i != j:
                multiplier = A[j][i] / A[i][i]
                for k in range(i, n):
                    A[j][k] -= multiplier*A[i][k]
                B[j] -= multiplier*B[i]

    for i in range(n):
        B[i] /= A[i][i]
    return B

In [132]:
def gauss_jordan_partial_pivoting(A, B):
    n = len(A)
    perm = [i for i in range(n)]
    for i in range(n):
        pivot = i
        for j in range(i + 1, n):
            if abs(A[perm[pivot]][i]) < abs(A[perm[j]][i]):
                pivot = j

        perm[pivot], perm[i] = perm[i], perm[pivot]
        for j in range(n):
            if i != j:
                multiplier = A[perm[j]][i] / A[perm[i]][i]
                for k in range(i, n):
                    A[perm[j]][k] -= multiplier*A[perm[i]][k]
                B[perm[j]] -= multiplier*B[perm[i]]

    for i in range(n):
        B[perm[i]] /= A[perm[i]][i]

    ans = [0 for _ in range(n)]
    for i in range(n):
        ans[i] = B[perm[i]]
    return ans


In [208]:
def gauss_jordan_complete_pivoting(A, B):
    n = len(A)
    perm_r = [i for i in range(n)]
    perm_c = perm_r[:]
    for i in range(n):
        pivot_r = i
        pivot_c = i
        for j in range(i, n):
            for k in range(i, n):
                if abs(A[perm_r[pivot_r]][perm_c[pivot_c]]) < abs(A[perm_r[j]][perm_c[k]]):
                    pivot_r = j
                    pivot_c = k

        perm_r[pivot_r], perm_r[i] = perm_r[i], perm_r[pivot_r]
        perm_c[pivot_c], perm_c[i] = perm_c[i], perm_c[pivot_c]
        for j in range(n):
            if i != j:
                multiplier = A[perm_r[j]][perm_c[i]] / A[perm_r[i]][perm_c[i]]
                for k in range(i, n):
                    A[perm_r[j]][perm_c[k]] -= multiplier*A[perm_r[i]][perm_c[k]]
                B[perm_r[j]] -= multiplier*B[perm_r[i]]

    for i in range(n):
        B[perm_r[i]] /= A[perm_r[i]][perm_c[i]]

    ans = [0 for _ in range(n)]
    for i in range(n):
        ans[perm_c[i]] = B[perm_r[i]]
    return ans


In [134]:
def euclid_dist(v1, v2):
    return sqrt(sum([pow(v1[i] - v2[i], 2) for i in range(len(v1))]))

In [227]:
count1 = 0
count2 = 0
count3 = 0

for i in range(100):
    A1 = random_matrix_size_n(50)
    B1 = random_vector_size_n(50)
    A2 = deepcopy(A1)
    B2 = deepcopy(B1)
    A3 = deepcopy(A1)
    B3 = deepcopy(B1)
    A4 = deepcopy(A1)
    B4 = deepcopy(B1)

    best_ind = 1
    vect1 = np.linalg.solve(A1, B1)
    vect2 = np.linalg.lstsq(A4, B4, rcond=None)

    v = gauss_jordan(A1, B1)
    d = euclid_dist(vect1, v)
    best = d

    v = gauss_jordan_partial_pivoting(A2, B2)
    d = euclid_dist(vect1, v)
    if d < best:
        best, best_ind = d, 2

    v = gauss_jordan_complete_pivoting(A3, B3)
    d = euclid_dist(vect1, v)
    if d < best:
        best, best_ind = d, 3

    if best_ind == 1:
        count1 += 1
    elif best_ind == 2:
        count2 += 1
    else:
        count3 += 1
    if i % 10 == 0:
        print(i)
print(count1)
print(count2)
print(count3)


0
10
20
30
40
50
60
70
80
90
0
40
60


In [18]:
def lu_decomposition_in_situ(A):
    n = len(A)
    for i in range(n - 1):
        for j in range(i + 1, n):
            multiplier = A[j][i] / A[i][i]
            A[j][i] = multiplier
            for k in range(i + 1, n):
                A[j][k] -= multiplier*A[i][k]
    return A

In [24]:
def compute_lu_distance(A, LU):
    C = np.zeros((len(A), len(A)))
    n = len(A)
    norm = 0
    for i in range(n):
        for j in range(n):
            entry = 0
            if i == j:
                for k in range(i):
                    entry += LU[i][k] * LU[k][i]
                entry += LU[i][i]
            elif i < j:
                for k in range(i):
                    entry += LU[i][k]*LU[k][j]
                entry += LU[i][j]
            else:
                for k in range(j + 1):
                    entry += LU[i][k]*LU[k][j]
            C[i][j] = entry
            norm += pow(A[i][j] - entry, 2)
    return sqrt(norm)

In [60]:
C1 = random_matrix_size_n(5)
C2 = deepcopy(C1)
lu_decomposition_in_situ(C1)
print(compute_lu_distance(C2, C1))

0.0002480984467303395
